In [31]:
import json, nltk, io, pickle
import numpy as np
from itertools import chain
from collections import defaultdict as dd

In [11]:
# this is just for debug reasons, I use diffrent dictionary

words = []
with io.open('/pio/data/data/reddit_sample/freqs', 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 20000:
            break
        words.append(line.split()[1])

words = set(words)

### Preprocess singles and make a dictionary

In [24]:
single = []
with io.open('/pio/data/data/reddit_sample/utterances.single.shuffled', 'r', encoding='utf-8') as f:
    for line in f:
        single.append(nltk.word_tokenize(line))

In [32]:
ws = dd(lambda: 0)
for s in single:
    for w in s:
        ws[w] += 1

In [44]:
ws_sorted = sorted(ws.items(), key=lambda x: -x[1])

In [67]:
voc = list(zip(*ws_sorted[:20000])[0])
voc.insert(0, '</s>')
voc.insert(0, '<s>')
voc.insert(0, '<unk>')

i_to_w = voc
w_to_i = {v:k for (k,v) in enumerate(voc)}
voc = set(voc)

In [77]:
with open('/pio/data/data/reddit_sample/wordlist.pkl', 'w') as f:
    pickle.dump(i_to_w, f)

In [68]:
single_cut = [[1] + [w_to_i.get(w, 0) for w in s] + [2] for s in single]

In [75]:
with open('/pio/data/data/reddit_sample/utterances.single.shuffled.pkl', 'w') as f:
    pickle.dump(single_cut, f)

In [106]:
freqs = [0] * len(voc)

for s in single_cut:
    for w in s:
        freqs[w] += 1

In [109]:
freqs = np.array(freqs, dtype=np.float32)
freqs /= sum(1 for s in single_cut for w in s)

In [115]:
with open('/pio/data/data/reddit_sample/freqs.pkl', 'w') as f:
    pickle.dump(freqs, f)

### Preprocess pairs

In [85]:
pairs = []
with io.open('/pio/data/data/reddit_sample/utterances.pairs.shuffled', 'r', encoding='utf-8') as f:
    for line in f:
        pairs.append(map(nltk.word_tokenize, line.split('\t')))

In [102]:
pairs_cut = [[[1] + [w_to_i.get(w, 0) for w in u] + [2] for u in s] for s in pairs]

In [104]:
with open('/pio/data/data/reddit_sample/utterances.pairs.shuffled.pkl', 'w') as f:
    pickle.dump(pairs_cut, f)

In [105]:
pairs_cut[0]

[[1, 342, 2], [1, 28, 40, 1918, 4, 2]]

### Make a train - test split

In [126]:
test_count = len(single_cut) / 10

In [127]:
train, test = single_cut[:-test_count], single_cut[-test_count:]

In [130]:
with open('/pio/data/data/reddit_sample/utterances.single.shuffled.train.pkl', 'w') as f:
    pickle.dump(train, f)
    
with open('/pio/data/data/reddit_sample/utterances.single.shuffled.test.pkl', 'w') as f:
    pickle.dump(test, f)

### Same with pairs

In [132]:
test_count_pairs = len(pairs_cut) / 20

In [133]:
train_pairs, test_pairs = pairs_cut[:-test_count_pairs], pairs_cut[-test_count_pairs:]

In [136]:
with open('/pio/data/data/reddit_sample/utterances.pairs.shuffled.train.pkl', 'w') as f:
    pickle.dump(train_pairs, f)
    
with open('/pio/data/data/reddit_sample/utterances.pairs.shuffled.test.pkl', 'w') as f:
    pickle.dump(test_pairs, f)